# Домашнее задание к лекции «Основы парсинга и работы с API»

## Задание 1.
### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/'
params = {
    'q': 'python'
}
# получение результатов запроса для 'python'
req = requests.get(URL, params=params)
req.text


In [ ]:
# парсинг полученного текста
soup = BeautifulSoup(req.text, 'html.parser')
soup

In [ ]:
# текст с блоками постов
post_bloks = soup.find_all('article', class_='post')
post_bloks

In [ ]:
# текст ссылок на посты
post_links_text = list(map(lambda x: x.find('a', class_='post__title_link'), post_bloks))
post_links_text

In [ ]:
# список ссылок
all_refs = list(map(lambda x: x.get('href'), post_links_text))
all_refs

In [ ]:
# оборачиваем в функцию получения списка ссылок
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/'

def get_links(URL, KEYWORDS):
    '''Возвращает список ссылок на посты из URL содержащих любое слово из KEYWORDS'''
    all_refs = []
    params = {}
    for word in KEYWORDS:
        params['q'] = word
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        post_bloks = soup.find_all('article', class_='post')
        all_refs += list(map(lambda x: x.find('a', class_='post__title_link').get('href'), post_bloks))
    return all_refs

all_links = get_links(URL, KEYWORDS)
all_links

In [ ]:
# даты
# заголовоки

def get_articles(all_links):
    '''По найденным ссылкам из поискового запроса возвращает Dataframe
    со столбцами'date', 'title', 'link' для найденных результатов'''
    df_articles = pd.DataFrame()
    for link in all_links:
        req = requests.get(link)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'))
        title = soup.find('span', class_='post__title-text').text
        row = {'date': date, 'title': title, 'link': link}
        df_articles = pd.concat([df_articles, pd.DataFrame([row])])
    return df_articles.reset_index(drop=True)
df_articles = get_articles(all_links)
df_articles

# Задание 2.¶
## Обязательная часть¶

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

## Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>



In [ ]:
EMAIL = {'emailAddresses': ['xxx@x.ru', 'yyy@y.com']}
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {"Vaar-Version": "0",
          "Vaar-Header-App-Product-Name": "hackcheck-web-avast",
          "Vaar-Header-App-Product": "hackcheck-web-avast"}
req = requests.post(URL, json=EMAIL, headers=headers)
req.text

In [ ]:
req.json()

In [ ]:
df = pd.DataFrame(req.json()['data'])
df.info()

In [ ]:
df
